### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7024,2023-11-27,Sérvia Primeira Liga,14:00,OKK Novi Pazar,Metalac,1.22,3.91,168.5,1.87,1.79,-10.5,2.02,1.71,CdJfYWUn,0.819672,0.255754,0.534759,0.558659,0.075427,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,73,91,1.93,2.94,0.000,0.000,0.741566,0.030912,0.117535,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7025,2023-11-27,Grécia Liga De Basquete,15:15,Olympiakos,Peristeri,1.09,7.10,157.5,1.82,1.85,-15.5,2.06,1.68,WtYZ21Q4,0.917431,0.140845,0.549451,0.540541,0.058276,94.154,18.144525,0.192711,2.4,1.341641,0.559017,114.33,1.100,0.112472,0.102247,90.0,107.202,21.439949,0.199996,2.4,1.341641,0.559017,97.01,1.324,0.233088,0.176048,60.0,111,89,1.03,1.09,0.000,0.000,1.037781,0.011560,0.143690,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7026,2023-11-27,Europa Liga Aba,16:00,Partizan,Borac,1.02,12.10,171.5,1.87,1.79,-19.5,1.96,1.75,Sdi9z4Cf,0.980392,0.082645,0.534759,0.558659,0.063037,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,91,96,1.36,1.61,0.000,0.000,1.194321,0.030912,0.080050,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7027,2023-11-27,Lituânia Lkl,13:50,Nevezis,Pieno Zvaigzdes,1.70,2.07,182.5,1.87,1.79,-3.5,2.09,1.66,tMIoUxeA,0.588235,0.483092,0.534759,0.558659,0.071327,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,74,88,3.30,2.98,0.000,0.000,0.138795,0.030912,0.162163,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7028,2023-11-27,México Lnbp,23:00,Abejas,Fuerza Regia,1.86,1.86,160.5,1.87,1.79,-1.5,2.01,1.72,zNrmWxqB,0.537634,0.537634,0.534759,0.558659,0.075269,156.154,37.826808,0.242240,1.2,1.643168,1.369306,209.44,2.064,0.636498,0.308381,-12.0,118.048,10.368998,0.087837,2.4,1.341641,0.559017,115.02,1.380,0.063640,0.046116,28.0,77,81,2.72,1.42,0.000,0.000,0.000000,0.030912,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7029,2023-11-27,México Lnbp,23:15,Astros,Libertadores,1.51,2.45,171.5,1.80,1.86,-5.5,2.09,1.66,SKNROfyo,0.662252,0.408163,0.555556,0.537634,0.070415,170.928,46.864919,0.274179,3.0,0.000000,0.000000,186.15,1.910,0.483167,0.252967,24.0,136.344,14.093407,0.103367,1.2,1.643168,1.369306,130.31,1.656,0.248254,0.149912,3.0,85,83,2.19,1.57,0.000,0.000,0.335697,0.023184,0.162163,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7030,2023-11-27,Romênia Divisão A,14:00,Brasov,Rapid Bucuresti,1.57,2.31,155.5,1.86,1.80,-4.5,2.01,1.72,hbk53dLj,0.636943,0.432900,0.537634,0.555556,0.069843,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,73,75,4.31,1.11,0.000,0.000,0.269721,0.023184,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7031,2023-11-27,Rússia Liga Vtb United,11:00,Astana,Samara,2.28,1.64,157.5,1.81,1.95,1.5,2.09,1.69,UHDoPefR,0.438596,0.609756,0.552486,0.512821,0.0483

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    
    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
7027,13:50,Lituânia Lkl,Nevezis,Pieno Zvaigzdes,1.70,Back Home
7028,23:00,México Lnbp,Abejas,Fuerza Regia,1.86,Back Home
7031,11:00,Rússia Liga Vtb United,Astana,Samara,2.28,Back Home
7032,11:30,Rússia Liga Vtb United,Uralmash Ekaterinburg,Enisey,2.12,Back Home
7035,23:00,Eua Ncaa,Fresno State,UC Santa Barbara,1.73,Back Home
7040,19:15,Uruguai Liga Uruguaia,Bigua,Nacional,2.05,Back Home
7041,21:00,Uruguai Liga Uruguaia,Urupan,Goes,1.68,Back Home
7042,21:30,Uruguai Liga Uruguaia,Hebraica y Macabi,Aguada,2.07,Back Home
7047,22:00,Eua Ncaa,TCU,Houston Christian,1.83,Back Home
7048,21:00,Eua Nba,Detroit Pistons,Washington Wizards,1.66,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
